Libs

In [204]:
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options as ChromeOptions
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
import re

Global variables

In [184]:
chrome_options = ChromeOptions()
chrome_options.add_argument("--headless=new")
main_df = pd.DataFrame(columns = ["Stadium", "Capacity", "Spectators", "Average",	"Matches", "sold out", "Capacity", "Club", "Year"])
seasons = [2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]
dfs = []
# 

Main function

In [185]:
def open_transfermarket():
  for i in range(len(seasons)):
    url = f'https://www.transfermarkt.com/saudi-pro-league/besucherzahlen/wettbewerb/SA1/saison_id/{seasons[i]}/plus/1'
    driver = webdriver.Chrome(options=chrome_options)
    driver.get(url)
    html = scrapingHTML(driver, i)
    #transform_data(html, i)
    print(f'Season {seasons[i]} - Scraping done!')

Function to get data with Selenium and BS4

In [186]:
def scrapingHTML(driver, i):
  try:
    WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it(driver.find_element(By.ID, "sp_message_iframe_851946")))
    driver.find_element(By.XPATH,"//*[@id='notice']/div[3]/div[3]/button").click()
  except NoSuchElementException as exc:
    pass
  
  table = driver.find_element(By.XPATH, '//*[@id="yw1"]/table')
  data = table.get_attribute('outerHTML')
  soup = BeautifulSoup(data, 'html.parser')
  html = soup.find(name='table')
  df = pd.read_html(str(html))[0]
  df["Club"] = ""
  df["Year"] = seasons[i]
  dfs.append(df)
  driver.quit()
  return html

Function to Transform data

In [187]:
def transform_data(html, i):
  df = pd.read_html(str(html))[0]
  df = df[df['#'] >= 1]
  df = df[["Stadium", "Capacity", "Spectators", "Average", "Matches", "sold out", "Capacity"]]
  df["Club"] = ""
  df["Year"] = seasons[i]

  ptn = "\B[ANH][Llab][-]?[a-zA-Z]+(?:-[a-zA-Z]+)?\s?[SFClub]{1,4}"
  
  for i in range(len(df)):
    if df.iloc[i, 0][0:23] == "Prince Mohammed bin Abd":
      df.iloc[i, 7] = "Al-Ansar/Ohod Club"
      df.iloc[i, 0] = "Prince Mohammed bin Abdul Aziz Stadium"
    elif df.iloc[i, 0] == "Damac Club StadiumDamac FC":
      df.iloc[i, 7] = "Damac FC"
      df.iloc[i, 0] = "Damac Club Stadium"
    else:
      df.iloc[i, 7] = re.findall(pattern=ptn, string=df.iloc[i, 0])[0]
      df.iloc[i, 0] = re.sub(pattern=ptn, string=df.iloc[i, 0], repl="")

  dfs.append(df)

Executando 

In [188]:
open_transfermarket()

Season 2007 - Scraping done!
Season 2008 - Scraping done!
Season 2009 - Scraping done!
Season 2010 - Scraping done!
Season 2011 - Scraping done!
Season 2012 - Scraping done!
Season 2013 - Scraping done!
Season 2014 - Scraping done!
Season 2015 - Scraping done!
Season 2016 - Scraping done!
Season 2017 - Scraping done!
Season 2018 - Scraping done!
Season 2019 - Scraping done!
Season 2020 - Scraping done!
Season 2021 - Scraping done!
Season 2022 - Scraping done!
Season 2023 - Scraping done!


In [197]:
df_og = pd.concat(dfs)


In [194]:
df_og = pd.concat(dfs)
df_og = df_og.reset_index()
df_og["index"] = df_og.index + 1
df_og['Matches'] = pd.to_numeric(df_og['Matches'], downcast='integer', errors='coerce')

In [206]:
df_og

,#,Stadium,Capacity,Spectators,Average,Matches,sold out,Capacity.1,Club,Year
0,1.0,King Abdullah Sports CityAl-Ittihad Club,62.345,27.0,27.000,1.0,-,43.3 %,,2007
1,NaN,King Abdullah Sports City,NaN,NaN,NaN,NaN,NaN,NaN,,2007
2,NaN,Al-Ittihad Club,NaN,NaN,NaN,NaN,NaN,NaN,,2007
3,2.0,King Fahd International StadiumAl-Hilal SFC,68.752,0.0,0.000,0.0,-,%,,2007
4,NaN,King Fahd International Stadium,NaN,NaN,NaN,NaN,NaN,NaN,,2007
...,...,...,...,...,...,...,...,...,...,...
50,NaN,Al-Okhdood Club,NaN,NaN,NaN,NaN,NaN,NaN,,2023
51,18.0,Damac Club StadiumDamac FC,5.000,2.227,1.114,2.0,-,22.3 %,,2023
52,NaN,Damac Club Stadium,NaN,NaN,NaN,NaN,NaN,NaN,,2023
53,NaN,Damac FC,NaN,NaN,NaN,NaN,NaN,NaN,,2023


In [205]:
df_og.to_csv("Arabic_Attendences_2007_2023.csv", encoding='utf-8', index=False)